# IO with Outcomes
1. analysis of a controlled example to stablish the effect of previous decisions on current optimal solutions

## IO with previous decisions
1. generating the true universal preference and time stamped optimal solutions
2. noising the optimal solutions as observations
3. Solving Existing IO to recover preference
4. Solving the true IO with time dependence for currect universal preference recovery

In [13]:
import numpy as np
from gurobipy import *

# Set the random seed for reproducibility
np.random.seed(42)

# Define the problem parameters
T = 5
theta_true = 1.0
u_tilde_true = [0.5] * T
u_hat = [0.5] * T
alpha = 0.5
epsilon = np.random.normal(0, 0.05, T)

# Solve the forward optimization problem to obtain the true optimal decisions
def solve_forward_problem(theta, u_tilde):
    x = [0] * T
    for t in range(T):
        model = Model("FO")
        x_t = model.addVar(lb=0, ub=1, name=f"x_{t}")
        if t == 0:
            x_prev = 0
        else:
            x_prev = x[t-1]
        model.setObjective(theta * (x_t - u_tilde[t])**2 + (x_t - x_prev)**2)
        model.optimize()
        x[t] = x_t.X
    return x

x_true = solve_forward_problem(theta_true, u_tilde_true)
x_hat = [x + e for x, e in zip(x_true, epsilon)]

# Solve the existing inverse optimization model
def solve_existing_IO_model():
    model = Model("Existing_IO")
    theta = model.addVar(lb=0, name="theta")
    x = [model.addVar(lb=0, ub=1, name=f"x_{t}") for t in range(T)]
    
    model.setObjective(quicksum((x_hat[t] - x[t])**2 for t in range(T)))
    
    for t in range(T):
        model.addConstr(x[t] == 0.5)
    
    model.optimize()
    theta_hat = theta.X
    return theta_hat

theta_hat_existing = solve_existing_IO_model()

# Solve the proposed inverse optimization model
def solve_proposed_IO_model(Theta_lb, Theta_ub):
    model = Model("Proposed_IO")
    theta = model.addVar(lb=Theta_lb, ub=Theta_ub, name="theta")
    u_tilde = [model.addVar(lb=0, ub=1, name=f"u_tilde_{t}") for t in range(T)]
    x = [model.addVar(lb=0, ub=1, name=f"x_{t}") for t in range(T)]
    
    model.setObjective(quicksum((x_hat[t] - x[t])**2 for t in range(T)))
    
    for t in range(T):
        if t == 0:
            x_prev = 0
        else:
            x_prev = x[t-1]
        model.addConstr((theta + 1) * x[t] == theta * u_tilde[t] + x_prev)
        if t < T-1:
            model.addConstr(u_tilde[t+1] == alpha * u_tilde[t] + (1 - alpha) * u_hat[t])
    model.params.NonConvex = 2 
    model.optimize()
    theta_hat = theta.X
    u_tilde_hat = [u.X for u in u_tilde]
    return theta_hat, u_tilde_hat

theta_hat_proposed, u_tilde_hat_proposed = solve_proposed_IO_model(0, 2)

# Print the results
print("True preference parameter: ", theta_true)
print("Estimated preference parameter (existing IO model): ", theta_hat_existing)
print("Estimated preference parameter (proposed IO model): ", theta_hat_proposed)
print("True unobservable features: ", u_tilde_true)
print("Estimated unobservable features (proposed IO model): ", u_tilde_hat_proposed)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 0 rows, 1 columns and 0 nonzeros
Model fingerprint: 0x13044a30
Model has 1 quadratic objective term
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [4e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective 1.25000000e-01
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 0 rows, 1 column